In [1]:
import numpy as np
import cv2
from detection.extract_features import extract_features
from detection.data import load_data

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
import time

def train_clf():
  features_vehicles = extract_features(vehicle_images, cspace='HSV')
  features_non_vehicles = extract_features(non_vehicle_images, cspace='HSV')
  # Create an array stack of feature vectors
  X = np.vstack((features_vehicles, features_non_vehicles)).astype(np.float64) 
  # Fit a per-column scaler
  X_scaler = StandardScaler().fit(X)
  # Apply the scaler to X
  scaled_X = X_scaler.transform(X)
  # Define the labels vector
  y = np.hstack((np.ones(len(features_vehicles)), np.zeros(len(features_non_vehicles))))
  # Split up data into randomized training and test sets
  rand_state = np.random.randint(0, 100)

  X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
  clf = AdaBoostClassifier(base_estimator=None, n_estimators=150, learning_rate=1.0, random_state=rand_state)

  # Check the training time for the SVC
  t=time.time()
  clf.fit(X_train, y_train)
  t2 = time.time()
  print(round(t2-t, 2), 'Seconds to train AdaBoostClassifier...')
  # Check the score of the SVC
  # Check the prediction time for a single sample
  t=time.time()
  n_predict_test = len(X_test)
  n_predict_train = len(X_train)
  print('Test Accuracy of AdaBoostClassifier = ', round(clf.score(X_test, y_test), 4))
  print('Train Accuracy of AdaBoostClassifier = ', round(clf.score(X_train, y_train), 4))
  # print('My AdaBoostClassifier predicts: ', clf.predict(X_test[0:n_predict]))
  # print('For these',n_predict_test, ' test labels: ', y_test[0:n_predict])
  t2 = time.time()
  print(round(t2-t, 5), 'Seconds to predict', n_predict_test,'test labels and', n_predict_train, 'test labels with AdaBoostClassifier')  

# train_clf()


In [3]:
from detection.extract_features import train_classifier

train_classifier()

/Users/kyledorman/Documents/carnd/vehicleDetection/data/full/vehicle.zip already downloaded.
/Users/kyledorman/Documents/carnd/vehicleDetection/data/full/non-vehicle.zip already downloaded.
extracting features from 17584 images
done extracting features from 17584 images
extracting features from 17936 images
done extracting features from 17936 images
About to train classifier
768.03 Seconds to train AdaBoost ...
Test Accuracy of AdaBoost = 0.9859
Train Accuracy of AdaBoost = 0.9918
7.45063 Seconds to predict 7104 test labels and 28416 test labels with AdaBoost
Saving classifier
Done saving classifier
saving scaler


AttributeError: 'DetectionConfig' object has no attribute 'sclaer_file'

In [ ]:
from detection.data import open_pickle_file
from detection.config import Config
from sklearn.model_selection import train_test_split

clf = open_pickle_file(Config().classifier_file("AdaBoost"))
scaler = open_pickle_file(Config().scaler_file())

vehicle_images, non_vehicle_images = load_data("full")

features_vehicles = extract_features(vehicle_images, cspace='HSV')
features_non_vehicles = extract_features(non_vehicle_images, cspace='HSV')

X = np.vstack((features_vehicles, features_non_vehicles)).astype(np.float64) 
y = np.hstack((np.ones(len(features_vehicles)), np.zeros(len(features_non_vehicles))))

scaled_X = X_scaler.transform(X)
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

def precision(y_data, y_pred, title):
  precision, recall, f1_score, label_count = precision_recall_fscore_support(y_data, y_pred)
  
  print(precision)
  print(recall)
  print(f1_score)
  
  plt.title("Precision(r), recall(g), f1_score(b) for " + title)
  plt.plot(range(len(precision)), precision, marker='o', color='r')
  plt.plot(range(len(precision)), recall, marker='o', color='g')
  plt.plot(range(len(precision)), f1_score, marker='o', color='b')
  plt.show()
  
precision(y_test, y_test_pred, "test data")
precision(y_train, y_train_pred, "train data")